In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.webgui import Draw as Drawgeo
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { width: 44em; }</style>"))

C:\Users\James\AppData\Local\Temp\ipykernel_10600\3909091748.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def generate_mesh(h, inner_rect_size, PML_size):
    
    inner_rect=WorkPlane().RectangleC(inner_rect_size,inner_rect_size).Face()
    scatterer = WorkPlane().Circle(0,0,1).Face()

    inner_rect.edges.name = 'innerbnd'
    scatterer.edges.name = 'scabnd'

    inner = inner_rect - scatterer

    #Drawgeo(inner)

    wp2=WorkPlane().RectangleC(inner_rect_size+inner_rect_size,inner_rect_size+inner_rect_size).RectangleC(inner_rect_size,inner_rect_size).Reverse()
    outer = wp2.Face()

    outer.edges.name = 'outerbnd'
    #inner.edges.name = 'innerbnd'
    inner.faces.name ='inner'
    outer.faces.name = 'pmlregion'

    #Drawgeo(Glue([outer,inner]))

    geo = OCCGeometry(Glue([inner, outer]), dim=2)
    mesh = Mesh(geo.GenerateMesh (maxh=h))
    mesh.Curve(5)
    return mesh

inner_rect_size = 10
PML_size = 5
h = 0.3
mesh = generate_mesh(h, inner_rect_size, PML_size)
Draw(mesh)




WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [3]:
K = CF((2*np.pi, 0))
phasor = exp(1j * ((K[0] * x) + (K[1] * y)))
ex = 1 * phasor
ey = 1 * phasor
ez =  CF((ex, ey))
Draw(ez, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Without PML:

In [4]:
fes = HCurl(mesh, order=4, complex=True, dirichlet='scabnd')
u = fes.TrialFunction()
v = fes.TestFunction()

omega = sqrt(K[0]**2 + K[1]**2)

a = BilinearForm(fes, symmetric=True)
a += curl(u)*curl(v)*dx - omega**2*u*v*dx
a += -1j * omega * u.Trace() * v.Trace() * ds('outerbnd')
#a += -1j*omega*(u)*(v)*ds("outerbnd")
a.Assemble()


b = LinearForm(fes)
b += SymbolicLFI(0)
b.Assemble()

scat = GridFunction(fes)
scat.Set(-ez, BND)

r = b.vec - a.mat * scat.vec
scat.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r

Draw(scat, mesh)
Draw(ez, mesh)
Draw(scat - ez, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## With PML

In [5]:
def absval(x):
    return sqrt(x**2)

d = inner_rect_size
z_x = IfPos(absval(x) - 2.5, x + (1j * ( absval(x) - d)/0.5) * x, x) # returns z_j if |x|>2.5 else returns x
z_y = IfPos(absval(y) - 2.5, y + (1j * ( absval(y) - d)/0.5) * y, y) # returns z_j if |y|>2.5 else returns y

Draw(z_x.imag, mesh)
Draw(z_y.imag, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [6]:
dzx = z_x.Diff(x)
dzy = z_y.Diff(y)

Draw(dzx.imag, mesh)
Draw(dzy.imag, mesh)

dz_tot = dzx * dzy
Draw(dz_tot.imag,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [7]:
Z = CF((dz_tot / (dzx**2), dz_tot / (dzy**2)))
#Z = CF((1,1))
#dz_tot = 1


def With_PML(mesh, p, K, ez, Z, dz_tot):

    fes = HCurl(mesh, order=p, complex=True, dirichlet='scabnd')
    u = fes.TrialFunction()
    v = fes.TestFunction()

    omega = sqrt(K[0]**2 + K[1]**2)

    #inner = CF( Z[0] * curl(u)[0] * curl(v)[0] + Z[1] * curl(u)[1] * curl(v)[1])
    inner = curl(u)*curl(v)


    a = BilinearForm(fes, symmetric=True)
    a += Z*inner * dx - dz_tot *omega**2*u*v * dx
    #a += -1j*omega*u*v * ds('outerbnd')
    a.Assemble()


    b = LinearForm(fes)
    b += SymbolicLFI(0)
    b.Assemble()

    scat_pml = GridFunction(fes)
    scat_pml.Set(-ez, BND)

    r = b.vec.CreateVector()
    r = b.vec - a.mat * scat_pml.vec
    scat_pml.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    return scat_pml, fes.ndof

scat_pml, _ = With_PML(mesh, 5, K, ez, Z, dz_tot)

Draw(scat_pml);
#Draw(ez, mesh)
#Draw(scat_pml - ez, mesh)

NgException: SymblicBFI needs scalar-valued CoefficientFunction

In [ ]:
"""
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from netgen.geom2d import SplineGeometry
from math import pi

geo = SplineGeometry()
geo.AddRectangle( (-0.1, -0.25), (0.1, 0.25), leftdomain=0, rightdomain=1, bc = "scat")
geo.AddCircle ( (0, 0), r=1.3, leftdomain=1, rightdomain=2)
geo.AddCircle ( (0, 0), r=1.4, leftdomain=2, rightdomain=0)
geo.SetMaterial(1, "air")


ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)

mesh.SetPML(pml.Radial(origin=(0,0), rad=1.4, alpha=0.1j), definedon=2)

mu0 = 4*pi*1e-7
eps0 = 8.854e-12

mu = CoefficientFunction(mu0)

freq=1000000000 #1000 MHz
w=2*pi*freq
k0=w*sqrt(mu0*eps0)
print("k0=",k0)


fes = HCurl(mesh, complex=True, order=5, dirichlet="scat")
u = fes.TrialFunction()
v = fes.TestFunction()

uin=CoefficientFunction( (0, exp(1J*k0*x)) )
uscat = GridFunction (fes)
uscat.Set (uin, definedon=mesh.Materials("air"))

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(curl(u)*curl(v) - k0*k0*u*v)
a += SymbolicBFI(-1J * k0 * u.Trace() * v.Trace(), BND)

f = LinearForm (fes)

a.Assemble()
f.Assemble()

res = uscat.vec.CreateVector()
res.data = f.vec - a.mat * uscat.vec
uscat.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky") * res


Draw (uin.real, mesh, "uin")
Draw (uscat.real, mesh, "uscat")
Draw (uin-uscat, mesh, "utot")
"""